In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

MODEL_NAME = "Salesforce/codet5p-220m"

# Load Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)  # ✅ Use Seq2Seq Model


c:\Users\musta\OneDrive\Desktop\AI-Powered Smart Query Assistant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Input: Natural Language Query
input_text = "Retrieve the top 10 customers by total purchase amount"

# Tokenize Input
inputs = tokenizer(input_text, return_tensors="pt")

# Generate SQL Query
output = model.generate(**inputs, max_length=100)
sql_query = tokenizer.decode(output[0], skip_special_tokens=True)

print(sql_query)



    // Create a new customer
    var customer = new Customer();
    customer.name = "New Customer";
    customer.email = "new-customer@gmail.com";
    customer.phone = "1234567890";
    customer.address = "New Street";
    customer.city = "New City";
    customer.state = "New State";
    customer.country = "New Country";
    customer.phone_


In [3]:
nl_query = "Find all orders placed in the last week."
input_text = f"Translate to SQL: {nl_query}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

output_ids = model.generate(input_ids, max_length=128)  
sql_query = tokenizer.decode(output_ids[0], skip_special_tokens=True)  

print("Generated SQL:", sql_query)  # Check if it's correct


Generated SQL: /*
 * Copyright (c) 2008-2021, Hazelcast, Inc. All Rights Reserved.
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS


In [4]:
from transformers import Trainer, TrainingArguments

In [5]:
from torch.utils.data import Dataset

class SQLDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = f"Translate to SQL: {item['input']}"
        output_text = item["output"]

        input_encodings = tokenizer(
            input_text,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )
        output_encodings = tokenizer(
            output_text,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )

        labels = output_encodings.input_ids.squeeze()
        labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in loss computation

        return {
            "input_ids": input_encodings.input_ids.squeeze(),
            "attention_mask": input_encodings.attention_mask.squeeze(),
            "labels": labels
        }
sample_data = [
    {
        "input": "Retrieve the top 10 customers by total purchase amount.",
        "output": "SELECT customer_name, SUM(purchase_amount) AS total FROM orders GROUP BY customer_name ORDER BY total DESC LIMIT 10;"
    },
    {
        "input": "Get the total number of products in the inventory.",
        "output": "SELECT COUNT(*) FROM products;"
    },
    {
        "input": "Find employees who joined after 2020.",
        "output": "SELECT * FROM employees WHERE join_date > '2020-01-01';"
    },
    {
        "input": "List all customers who made purchases in the last 30 days.",
        "output": "SELECT DISTINCT customer_name FROM orders WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY);"
    },
    {
        "input": "Retrieve all orders placed by a specific customer named John Doe.",
        "output": "SELECT * FROM orders WHERE customer_name = 'John Doe';"
    },
    {
        "input": "Get the average order value from the orders table.",
        "output": "SELECT AVG(order_amount) FROM orders;"
    },
    {
        "input": "Show the 5 most recent transactions.",
        "output": "SELECT * FROM transactions ORDER BY transaction_date DESC LIMIT 5;"
    },
    {
        "input": "Find the highest salary among all employees.",
        "output": "SELECT MAX(salary) FROM employees;"
    },
    {
        "input": "List the names and emails of all customers.",
        "output": "SELECT name, email FROM customers;"
    },
    {
        "input": "Retrieve the total revenue generated this month.",
        "output": "SELECT SUM(amount) FROM transactions WHERE transaction_date >= DATE_FORMAT(CURDATE(), '%Y-%m-01');"
    }
]

# Create dataset
dataset = SQLDataset(sample_data, tokenizer)


In [6]:

!pip install -U transformers accelerate



In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./sql_model_finetuned",
    evaluation_strategy="no",  # <-- Change this to disable evaluation
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    # eval_dataset=eval_dataset  # <-- Add this
)


# Start training
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=15, training_loss=1.3230331420898438, metrics={'train_runtime': 184.9332, 'train_samples_per_second': 0.27, 'train_steps_per_second': 0.081, 'total_flos': 7611973632000.0, 'train_loss': 1.3230331420898438, 'epoch': 5.0})

In [13]:
def generate_sql(nl_query):
    input_text = f"Translate to SQL: {nl_query}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(input_ids, max_length=128)  
    sql_query = tokenizer.decode(output_ids[0], skip_special_tokens=True)  

    return sql_query

# Sample Test
test_queries = [
    "Retrieve the top 10 customers by total purchase amount.",
    "Get the total number of products in inventory.",
    "Find employees who joined after 2021."
]

for query in test_queries:
    print(f"Query: {query}")
    print(f"Generated SQL: {generate_sql(query)}\n")


Query: Retrieve the top 10 customers by total purchase amount.
Generated SQL: SELECT customer_name FROM orders WHERE order_date >= DATE_SUB(CURDATE(), SUB(CURDATE(), INTERVAL 10 DAY));

Query: Get the total number of products in inventory.
Generated SQL: SELECT COUNT(*) FROM products;

Query: Find employees who joined after 2021.
Generated SQL: 
SELECT * FROM employees WHERE joined_date >= DATE_SUB(CURDATE(), INTERVAL INTERVAL 30 DAY);



In [16]:
!pip install datasets


   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.2 MB ? eta -:--:--
    --------------------------------------- 0.5/25.2 MB 2.0 MB/s eta 0:00:13
   - -------------------------------------- 0.8/25.2 MB 1.2 MB/s eta 0:00:21
   - -------------------------------------- 1.0/25.2 MB 1.3 MB/s eta 0:00:19
   -- ------------------------------------- 1.3/25.2 MB 1.3 MB/s eta 0:00:18
   -- ------------------------------------- 1.6/25.2 MB 1.4 MB/s eta 0:00:18
   -- ------------------------------------- 1.8/25.2 MB 1.3 MB/s eta 0:00:19
   --- ------------------------------------ 2.4/25.2 MB 1.4 MB/s eta 0:00:17
   ---- ----------------------------------- 2.6/25.2 MB 1.4 MB/s eta 0:00:17
   ---- ----------------------------------- 3.1/25.2 MB 1.6 MB/s eta 0:00:15
   ----- ---------------------------------- 3.4/25.2 MB 1.6 MB/s eta 0:00:14
   ------ ----------

In [18]:
!pip install --upgrade datasets


In [19]:
from datasets import load_dataset

dataset = load_dataset("wikisql", trust_remote_code=True)


Generating train split: 100%|██████████| 56355/56355 [00:06<00:00, 8783.75 examples/s] 


In [20]:
from datasets import load_dataset

dataset = load_dataset("wikisql", trust_remote_code=True)
train_dataset = dataset["train"]
val_dataset = dataset["validation"]


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Salesforce/codet5p-220m"  # Change to another model if needed
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)




In [23]:
def preprocess_function(examples):
    inputs = [f"translate SQL to English: {query}" for query in examples["question"]]
    
    # Extract "human_readable" text for each SQL query
    targets = [sql["human_readable"] for sql in examples["sql"]]
    
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)


Map: 100%|██████████| 8421/8421 [00:00<00:00, 11691.21 examples/s]


In [24]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./sql_to_natural_language_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
)


c:\Users\musta\OneDrive\Desktop\AI-Powered Smart Query Assistant\.venv\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

# Data collator to pad sequences dynamically
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


C:\Users\musta\AppData\Local\Temp\ipykernel_38876\334702887.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [26]:
model.save_pretrained("./sql_to_natural_language_model")
tokenizer.save_pretrained("./sql_to_natural_language_model")


('./sql_to_natural_language_model\\tokenizer_config.json',
 './sql_to_natural_language_model\\special_tokens_map.json',
 './sql_to_natural_language_model\\vocab.json',
 './sql_to_natural_language_model\\merges.txt',
 './sql_to_natural_language_model\\added_tokens.json',
 './sql_to_natural_language_model\\tokenizer.json')

In [27]:
metrics = trainer.evaluate()
print(metrics)



{'eval_loss': 12.06804084777832, 'eval_model_preparation_time': 0.0068, 'eval_runtime': 5511.0255, 'eval_samples_per_second': 1.528, 'eval_steps_per_second': 0.191}


In [29]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
model_path = "./sql_to_natural_language_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Define the inference pipeline
nlp_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Test with a sample SQL query
sql_query = "SELECT name FROM employees WHERE salary > 50000;"
input_text = f"translate SQL to English: {sql_query}"

# Generate output
output = nlp_pipeline(input_text, max_length=128)
print(output[0]['generated_text'])


Device set to use cpu


SELECT name FROM employees WHERE salary > 50000;


In [30]:
# Generate output
output = nlp_pipeline(input_text, max_length=128)
print(output[0]['generated_text'])

SELECT name FROM employees WHERE salary > 50000;


In [36]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned model and tokenizer
model_path = "./sql_to_natural_language_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)

# Define the inference pipeline
nlp_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0 if device.type == "cuda" else -1)

# Test with a sample SQL query
sql_query = "SELECT name FROM employees WHERE salary > 50000;"
input_text = f"translate SQL to English: {sql_query}"

# Generate output
output = nlp_pipeline(input_text, max_length=128)
print(output[0]['generated_text'])

Device set to use cpu


SELECT name FROM employees WHERE salary > 50000;


In [35]:
# from transformers import T5ForConditionalGeneration, AutoTokenizer

# checkpoint = "Salesforce/codet5p-220m"
# device = "cuda" # for GPU usage or "cpu" for CPU usage

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = T5ForConditionalGeneration.from_pretrained(checkpoint).to(device)

# inputs = tokenizer.encode("def print_hello_world():<extra_id_0>", return_tensors="pt").to(device)
# outputs = model.generate(inputs, max_length=10)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# # ==> print "Hello World"


In [37]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available!")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available. Using CPU.")

GPU is not available. Using CPU.


In [38]:
# Move model to GPU
model.to(device)

# Move inputs to GPU
inputs = tokenizer(input_text, return_tensors="pt").to(device)

In [42]:
!pip uninstall torch torchvision torchaudio

^C


In [40]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

GPU available: False
GPU name: None


In [41]:
!nvidia-smi

Fri Mar 14 13:07:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2050      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   57C    P0              4W /   42W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
C:\Users\musta\OneDrive\Desktop\AI-Powered Smart Query Assistant